In [1]:
import os
import time
import cv2
import numpy as np
from model.yolo_model import YOLO

In [2]:
def process_image(img):
    """Resize, reduce and expand image.

    # Argument:
        img: original image.

    # Returns
        image: ndarray(64, 64, 3), processed image.
    """
    image = cv2.resize(img, (416, 416),
                       interpolation=cv2.INTER_CUBIC)
    image = np.array(image, dtype='float32')
    image /= 255.
    image = np.expand_dims(image, axis=0)

    return image

In [3]:
def get_classes(file):
    """Get classes name.

    # Argument:
        file: classes name for database.

    # Returns
        class_names: List, classes name.

    """
    with open(file) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]

    return class_names

In [4]:
def draw(image, boxes, scores, classes, all_classes):
    """Draw the boxes on the image.

    # Argument:
        image: original image.
        boxes: ndarray, boxes of objects.
        classes: ndarray, classes of objects.
        scores: ndarray, scores of objects.
        all_classes: all classes name.
    """
    for box, score, cl in zip(boxes, scores, classes):
        x, y, w, h = box

        top = max(0, np.floor(x + 0.5).astype(int))
        left = max(0, np.floor(y + 0.5).astype(int))
        right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
        bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))

        cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
        cv2.putText(image, '{0} {1:.2f}'.format(all_classes[cl], score),
                    (top, left - 6),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (0, 0, 255), 1,
                    cv2.LINE_AA)

        print('class: {0}, score: {1:.2f}'.format(all_classes[cl], score))
        print('box coordinate x,y,w,h: {0}'.format(box))
    print()    

In [5]:
def detect_image(image, yolo, all_classes):
    """Use yolo v3 to detect images.

    # Argument:
        image: original image.
        yolo: YOLO, yolo model.
        all_classes: all classes name.

    # Returns:
        image: processed image.
    """
    pimage = process_image(image)

    start = time.time()
    boxes, classes, scores = yolo.predict(pimage, image.shape)
    end = time.time()

    print('time: {0:.2f}s'.format(end - start))

    if boxes is not None:
        draw(image, boxes, scores, classes, all_classes)

    return image

In [6]:
def detect_video(video, yolo, all_classes):
    """Use yolo v3 to detect video.

    # Argument:
        video: video file.
        yolo: YOLO, yolo model.
        all_classes: all classes name.
    """
    video_path = os.path.join("C:\\Users\\Chandhinii.B\\Desktop\\videos\\test\\", video)
    camera = cv2.VideoCapture(video_path)
    cv2.namedWindow("detection", cv2.WINDOW_AUTOSIZE)

    # Prepare for saving the detected video
    sz = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fourcc = cv2.VideoWriter_fourcc(*'mpeg')

    
    vout = cv2.VideoWriter()
    vout.open(os.path.join("C:\\Users\\Chandhinii.B\\res\\", video), fourcc, 20, sz, True)

    while True:
        res, frame = camera.read()

        if not res:
            break
        
        image = detect_image(frame, yolo, all_classes)
        cv2.imshow("detection", image)

        # Save the video frame by frame
        vout.write(image)

        if cv2.waitKey(110) & 0xff == 27:
                break

    vout.release()
    camera.release()
    

In [7]:
yolo = YOLO(0.6, 0.5)
file = 'data/coco_classes.txt'
all_classes = get_classes(file)

## Detecting images

In [8]:
f = 'person.jpg'
path = "C:\\Users\\Chandhinii.B\\Desktop\\images\\test\\"+f
image = cv2.imread(path)
image = detect_image(image, yolo, all_classes)
cv2.imwrite("C:\\Users\\Chandhinii.B\\res\\" + f, image)

1/1 [==============================] - 3s 3s/step
time: 2.96s
class: person, score: 1.00
box coordinate x,y,w,h: [187.70118713  84.55980635  91.62084579 304.26396179]
class: horse, score: 1.00
box coordinate x,y,w,h: [396.53278351 137.27252269 215.59455872 208.5619216 ]
class: dog, score: 1.00
box coordinate x,y,w,h: [ 61.35536671 263.36689949 145.4764843   88.21137941]



True

## Detecting on video

In [9]:
# detect videos one at a time in videos/test folder    
video = 'dogduck.mp4'
detect_video(video, yolo, all_classes)

1/1 [==============================] - 0s 467ms/step
time: 0.54s
class: dog, score: 0.92
box coordinate x,y,w,h: [  9.88223553 261.96012497 337.10035515 318.67458344]

1/1 [==============================] - 0s 451ms/step
time: 0.53s
class: dog, score: 0.82
box coordinate x,y,w,h: [  6.94701672 256.69948578 345.40075111 316.44325256]

1/1 [==============================] - 0s 412ms/step
time: 0.48s
class: dog, score: 0.81
box coordinate x,y,w,h: [  5.75543118 255.53184509 347.29523659 320.45898438]

1/1 [==============================] - 0s 416ms/step
time: 0.48s
1/1 [==============================] - 0s 428ms/step
time: 0.49s
1/1 [==============================] - 0s 416ms/step
time: 0.48s
class: dog, score: 0.85
box coordinate x,y,w,h: [ 36.99435234 209.25127029 326.90922928 317.0822525 ]

1/1 [==============================] - 0s 390ms/step
time: 0.45s
class: dog, score: 0.96
box coordinate x,y,w,h: [ 36.7350502  198.42737198 328.89586258 335.71720123]

1/1 [=========================

1/1 [==============================] - 0s 476ms/step
time: 0.56s
class: dog, score: 0.88
box coordinate x,y,w,h: [ 97.07326698 139.73269463 172.37052536 249.57410812]
class: dog, score: 0.63
box coordinate x,y,w,h: [ 48.38574982 146.8806839  310.15137482 376.70585632]
class: bird, score: 0.61
box coordinate x,y,w,h: [ 44.41565132 173.84140015 318.39638138 364.48501587]

1/1 [==============================] - 0s 479ms/step
time: 0.57s
class: dog, score: 0.91
box coordinate x,y,w,h: [ 95.74065876 139.94628906 174.20150185 251.19403839]
class: dog, score: 0.66
box coordinate x,y,w,h: [ 48.64181042 151.25745773 309.03999901 368.30165863]

1/1 [==============================] - 0s 466ms/step
time: 0.55s
class: dog, score: 0.86
box coordinate x,y,w,h: [ 93.09527969 140.38602829 177.01337433 247.66040802]
class: dog, score: 0.65
box coordinate x,y,w,h: [ 49.42367172 152.35801697 307.14574432 365.6413269 ]

1/1 [==============================] - 0s 468ms/step
time: 0.55s
class: dog, score: 0.8

1/1 [==============================] - 0s 459ms/step
time: 0.53s
class: dog, score: 0.66
box coordinate x,y,w,h: [ 92.17342377 134.36220169 177.20069122 265.05847931]
class: dog, score: 0.62
box coordinate x,y,w,h: [ 50.83135986 132.53379822 311.89702225 402.4256897 ]
class: bird, score: 0.87
box coordinate x,y,w,h: [ 48.59776115 165.03686905 318.13869476 381.98421478]

1/1 [==============================] - 0s 480ms/step
time: 0.57s
class: bird, score: 0.95
box coordinate x,y,w,h: [ 49.64030933 164.48516846 315.21449089 383.4690094 ]

1/1 [==============================] - 0s 449ms/step
time: 0.53s
class: dog, score: 0.67
box coordinate x,y,w,h: [ 47.40015411 133.41976166 318.56733322 399.0221405 ]
class: bird, score: 0.83
box coordinate x,y,w,h: [ 45.8769455  165.77213287 322.55327225 381.3633728 ]

1/1 [==============================] - 0s 458ms/step
time: 0.54s
class: bird, score: 0.85
box coordinate x,y,w,h: [ 46.3653059  167.36604691 323.06186867 377.6556778 ]

1/1 [=============

1/1 [==============================] - 1s 506ms/step
time: 0.61s
class: bird, score: 1.00
box coordinate x,y,w,h: [ 56.85477924  89.31388855 286.60959816 543.35922241]

1/1 [==============================] - 0s 496ms/step
time: 0.59s
class: bird, score: 1.00
box coordinate x,y,w,h: [ 57.13780022  88.55541229 287.62332916 545.33554077]

1/1 [==============================] - 0s 485ms/step
time: 0.57s
class: bird, score: 1.00
box coordinate x,y,w,h: [ 52.3172226   90.44698715 291.20970917 485.98575592]

1/1 [==============================] - 0s 469ms/step
time: 0.56s
class: bird, score: 1.00
box coordinate x,y,w,h: [ 49.41083145  87.17988968 297.78145218 490.91739655]

1/1 [==============================] - 1s 528ms/step
time: 0.62s
class: bird, score: 1.00
box coordinate x,y,w,h: [ 47.41687584  81.83908463 299.16421509 497.09499359]

1/1 [==============================] - 0s 459ms/step
time: 0.55s
class: bird, score: 1.00
box coordinate x,y,w,h: [ 45.07611752  70.81260681 300.98679924 5

1/1 [==============================] - 1s 587ms/step
time: 0.68s
class: bird, score: 0.99
box coordinate x,y,w,h: [ 91.77382278 126.05241776 264.32994461 406.99214935]

1/1 [==============================] - 1s 569ms/step
time: 0.66s
class: bird, score: 1.00
box coordinate x,y,w,h: [ 94.47327995 125.48593521 261.55535507 411.30268097]

1/1 [==============================] - 0s 466ms/step
time: 0.56s
class: bird, score: 1.00
box coordinate x,y,w,h: [ 95.27402401 130.39234161 260.65345192 401.96052551]

1/1 [==============================] - 0s 486ms/step
time: 0.57s
class: bird, score: 0.98
box coordinate x,y,w,h: [ 96.18500137 130.48171997 259.4187088  397.98233032]

1/1 [==============================] - 0s 475ms/step
time: 0.56s
class: bird, score: 0.99
box coordinate x,y,w,h: [ 94.64722157 127.53017426 262.49116325 402.91770935]

1/1 [==============================] - 0s 458ms/step
time: 0.55s
class: bird, score: 0.99
box coordinate x,y,w,h: [ 94.64801884 125.23170471 260.88105202 4

1/1 [==============================] - 1s 563ms/step
time: 0.69s
class: bird, score: 0.89
box coordinate x,y,w,h: [ 40.91592216 112.89287567 313.53092575 521.24504089]

1/1 [==============================] - 1s 532ms/step
time: 0.65s
class: bear, score: 0.64
box coordinate x,y,w,h: [154.82518673 150.20969391 194.94365883 228.48827362]
class: bird, score: 0.82
box coordinate x,y,w,h: [ 41.05475235 111.04736328 312.5235939  523.19473267]

1/1 [==============================] - 1s 576ms/step
time: 0.67s
class: bird, score: 0.96
box coordinate x,y,w,h: [ 45.07214165 114.34885025 306.31993866 523.42449188]

1/1 [==============================] - 1s 590ms/step
time: 0.72s
class: bird, score: 0.90
box coordinate x,y,w,h: [ 40.96154499 107.62674332 315.05516243 529.20322418]

1/1 [==============================] - 1s 576ms/step
time: 0.70s
class: bird, score: 0.98
box coordinate x,y,w,h: [ 41.58635426  98.0872345  314.53038979 547.60131836]

1/1 [==============================] - 1s 545ms/step